In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
import pandas as pd

In [11]:
target_X= pd.read_csv("../data/tem_train_input_7.csv").iloc[:,1:].values.astype(np.float32)
target_y =pd.read_csv("../data/tem_train_output_7.csv").iloc[:,1:].values.astype(np.float32)
test_X= pd.read_csv("../data/tem_val_input_7.csv").iloc[:,1:].values.astype(np.float32)
test_y =pd.read_csv("../data/tem_val_output_7.csv").iloc[:,1:].values.astype(np.float32)

X_train=target_X.astype(np.float32)
y_train=target_y.astype(np.float32)
#y_train.astype(np.float32)
backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

In [13]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]

#################################################################################
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 unit
    # 4 nhead
    # 5 nlayers
    # dropout
    return X_train.shape[1],y_train.shape[1],128,1,4,0.1

#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_train,y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models_G(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model_G(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[select]
        y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_bootstrap, y_bootstrap, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################
# SMAPE 용
def train_bagging_models_smape(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[select]
        y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_bootstrap, y_bootstrap, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

##########################################################################################
# 트랜스퍼 레이어
class PositionalEncoding(layers.Layer):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = layers.Dropout(rate=dropout)

        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = np.zeros((max_len, d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        pe = pe[np.newaxis, ...]

        self.pe = tf.constant(pe, dtype=tf.float32)

    def call(self, x):
        x = x + self.pe[:, :tf.shape(x)[1], :]
        return self.dropout(x)
##########################################################################################
# 트랜스퍼 레이어
def create_model(fn,d_model, nlayers, nhead, dropout, iw, ow,lr):
    
    
    x = layers.Dense(d_model // 2, activation='relu')(pretrained_output_reshaped)
    x = layers.Dense(d_model, activation='relu')(x)
    
    pos_encoding = PositionalEncoding(d_model, dropout)
    x = pos_encoding(x)
    
    for _ in range(nlayers):
        attn_output = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model, dropout=dropout)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = layers.Dense(d_model, activation='relu')(x)
        ffn_output = layers.Dense(d_model)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_output)
    
    x = layers.Dense(d_model // 2, activation='relu')(x)
    x = layers.Dense(1)(x)
    x = tf.squeeze(x, axis=-1)
    
    outputs = layers.Dense((iw + ow) // 2, activation='relu')(x)
    outputs = layers.Dense(ow)(outputs)
    
    optimizer = Adam(learning_rate=lr)
    target_model = Model(inputs=inputs, outputs=outputs)
    target_model.compile(optimizer=optimizer, loss=fn)
    
    return target_model

#################################################################################
# 트랜스포머 모델 생성 함수
def bulid_model(iw, ow, d_model, nhead, nlayers, dropout=0.1):
    inputs = tf.keras.Input(shape=(iw, 1))
    x = layers.Dense(d_model // 2, activation='relu')(inputs)
    x = layers.Dense(d_model, activation='relu')(x)

    pos_encoding = PositionalEncoding(d_model, dropout)
    x = pos_encoding(x)

    for _ in range(nlayers):
        attn_output = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model, dropout=dropout)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = layers.Dense(d_model, activation='relu')(x)
        ffn_output = layers.Dense(d_model)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_output)

    x = layers.Dense(d_model // 2, activation='relu')(x)
    x = layers.Dense(1)(x)
    x = tf.squeeze(x, axis=-1)

    outputs = layers.Dense((iw + ow) // 2, activation='relu')(x)
    outputs = layers.Dense(ow)(outputs)

    model = Model(inputs=inputs, outputs=outputs)
    return model

#################################################################################
# 부트스트랩 샘플링
# 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr=0.001):
    models = {}
    iw, ow, d_model, nhead, nlayers, dropout = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(iw, ow, d_model, nhead, nlayers, dropout=0.5)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys
#################################################################################
# 예측

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions



In [16]:
model_num = 5


mase_models = train_bagging_models(model_num, MASE(y_train,24),2000,10,8,lr=0.001)
mape_models = train_bagging_models(model_num,'mape',2000,10,8,lr=0.001)
smape_models = train_bagging_models(model_num, SMAPE(),2000,10,8,lr=0.001)
mae_models = train_bagging_models(model_num, 'mae',2000,10,8,lr=0.001)
mse_models = train_bagging_models(model_num, 'mse',2000,10,8,lr=0.001)

'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'#######################################

In [21]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions = bagging_predict2(pred1, test_X)
mase_predictions = bagging_predict2(pred2, test_X)
mape_predictions = bagging_predict2(pred3, test_X)
mae_predictions = bagging_predict2(pred4, test_X)
mse_predictions = bagging_predict2(pred5, test_X)

12/12 [==============================] - 0s 40ms/step


In [29]:
concat = np.concatenate([smape_predictions, mase_predictions,mape_predictions,mae_predictions,mse_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
pd.DataFrame(fin_pred.flatten()).to_csv("../result7/transformer.csv")

#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(2.9242463, 2.230567)

In [31]:
concat = np.concatenate([mae_predictions, mase_predictions,mse_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
pd.DataFrame(fin_pred.flatten()).to_csv("../best7/transformer.csv")

#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(2.7340772, 2.0222194)

In [23]:
concat = np.concatenate([smape_predictions, mase_predictions,mape_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(6.699875, 4.246545)

In [24]:
concat = np.concatenate([smape_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(9.441889, 5.1176224)

In [25]:
concat = np.concatenate([mape_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(17.974054, 15.377315)

In [26]:
concat = np.concatenate([mase_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(2.8801794, 2.1755111)

In [27]:
concat = np.concatenate([mae_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(2.7922385, 2.0875342)

In [28]:
concat = np.concatenate([mse_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(2.7685616, 2.0499594)

In [18]:
pd.DataFrame(fin_pred).to_csv("../result7_new/transformer/pred_mid.csv")
for i in range(10):
    pd.DataFrame(concat[i]).to_csv(f"../result7_new/transformer/pred{i}.csv")